In [15]:
import numpy as np
from collections import Counter

In [2]:
fp = 'malba_dataset_completo_ficha_tecnica_sin_formato.npy'

In [47]:
data = np.load(fp,allow_pickle=True)

In [48]:
data[0]

{'ficha_tecnica': 'Duración interna, 2018 - Ficha técnica - Titulo: Duración interna - Año: 2018 - Técnica: Madera, pintura, cartas - 20,5 x 118 x 10 cm - Nro. de inventario: 2018.08 - Donación: Del artista y de la galería Isla Flotante, 2018 - Fuera de exposición',
 'link_obra': 'https://coleccion.malba.org.ar/artistas/accinelli-pablo/',
 'texto': '',
 'link_autor': 'https://coleccion.malba.org.ar/artistas/accinelli-pablo/'}

In [49]:
fichas_tecnicas = {o['link_obra']:o['ficha_tecnica'] for o in data}

In [123]:
tokens_txt_fichas_tecnicas = ''.join(fichas_tecnicas.values()).split(' ')

In [51]:
f = data[0]['ficha_tecnica']

In [52]:
info = f.split('-')

In [53]:
info[2:]

[' Titulo: Duración interna ',
 ' Año: 2018 ',
 ' Técnica: Madera, pintura, cartas ',
 ' 20,5 x 118 x 10 cm ',
 ' Nro. de inventario: 2018.08 ',
 ' Donación: Del artista y de la galería Isla Flotante, 2018 ',
 ' Fuera de exposición']

In [143]:
def get_structured_info(ficha_tecnica):
    structured_info = {}
    info = ficha_tecnica.split('-')
    for line in info[2:]:
        if ':' in line:
            column, value = line.strip().split(':',maxsplit=1)
            structured_info[column] = value.strip()
        elif 'x' in line and any(c.isdigit() for c in line):
            structured_info['Medidas'] = line
        elif 'En exposición' in line:
            structured_info['En exposición'] = True
            break
        elif 'Fuera de exposición' in line:
            structured_info['En exposición'] = False
            break
    return structured_info

In [144]:
data[1]['ficha_tecnica']

'Prehistoria, 2005 - Ficha técnica - Titulo: Prehistoria - Año: 2005 - Técnica: Animación en video Edición 3 de 5 + 1 P/A - Duración: 29 minutos 33 segundos - Nro. de inventario: 2006.02 - Donación: Adquisición gracias al aporte de Sergio Quattrini y Fundación Eduardo F. Costantini, 2006 - Fuera de exposición'

In [145]:
get_structured_info(data[1]['ficha_tecnica'])

{'Titulo': 'Prehistoria',
 'Año': '2005',
 'Técnica': 'Animación en video Edición 3 de 5 + 1 P/A',
 'Duración': '29 minutos 33 segundos',
 'Nro. de inventario': '2006.02',
 'Donación': 'Adquisición gracias al aporte de Sergio Quattrini y Fundación Eduardo F. Costantini, 2006',
 'En exposición': False}

In [158]:
obras_clean = []
for obra in data:
    structured_info = get_structured_info(obra['ficha_tecnica'])
    obra_clean = {
        'URL_obra': obra['link_obra'],
        'Texto': obra['texto'] if obra['texto'] else None,
        'URL_autor': obra['link_autor']
    }
    obra_clean_final = dict(obra_clean, **structured_info)
    obras_clean.append(obra_clean_final)

In [159]:
import pandas as pd

In [160]:
obras_clean[:2]

[{'URL_obra': 'https://coleccion.malba.org.ar/artistas/accinelli-pablo/',
  'Texto': None,
  'URL_autor': 'https://coleccion.malba.org.ar/artistas/accinelli-pablo/',
  'Titulo': 'Duración interna',
  'Año': '2018',
  'Técnica': 'Madera, pintura, cartas',
  'Medidas': ' 20,5 x 118 x 10 cm ',
  'Nro. de inventario': '2018.08',
  'Donación': 'Del artista y de la galería Isla Flotante, 2018',
  'En exposición': False},
 {'URL_obra': 'https://coleccion.malba.org.ar/artistas/acevedo-velarde-gabriel/',
  'Texto': None,
  'URL_autor': 'https://coleccion.malba.org.ar/artistas/acevedo-velarde-gabriel/',
  'Titulo': 'Prehistoria',
  'Año': '2005',
  'Técnica': 'Animación en video Edición 3 de 5 + 1 P/A',
  'Duración': '29 minutos 33 segundos',
  'Nro. de inventario': '2006.02',
  'Donación': 'Adquisición gracias al aporte de Sergio Quattrini y Fundación Eduardo F. Costantini, 2006',
  'En exposición': False}]

In [161]:
df_obras = pd.DataFrame(obras_clean)

In [167]:
df_obras_clean = df_obras.dropna(axis=1,thresh=15)

In [168]:
df_obras_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   URL_obra            569 non-null    object
 1   Texto               277 non-null    object
 2   URL_autor           569 non-null    object
 3   Titulo              569 non-null    object
 4   Año                 569 non-null    object
 5   Técnica             569 non-null    object
 6   Medidas             251 non-null    object
 7   Nro. de inventario  569 non-null    object
 8   Donación            547 non-null    object
 9   En exposición       569 non-null    bool  
 10  Duración            18 non-null     object
dtypes: bool(1), object(10)
memory usage: 45.1+ KB


In [169]:
df_obras_clean.to_csv('datos_estructurados_malba',index=False)